# Russian Wikipedia network analisys

### This script provides a descriptive analysis of the Russian Wikipedia directed network
- Nodes of the network: links to Wikipedia pages.
- Edges of the network: edge from node $A$ to node $B$ means that page $A$ contains a link to page $B$

In [1]:
import pickle
import random
import numpy as np
import networkx as nx

In [3]:
GRAPH_PATH = './../saved_graphs/WIKI_50000.pkl'
with open(GRAPH_PATH, 'rb') as f:
    G = pickle.load(f)

In [4]:
def print_nodes(G, num_of_links=5, display_links=False):
    random_nodes10 = [random.choice(list(G.nodes.keys())) for _ in range(num_of_links)]
    for link in random_nodes10: print(G.nodes[link]['name'].split('—')[0][:-1], link*display_links)

In [5]:
print_nodes(G, num_of_links=5, display_links=False)

Источники книг 
SLC-41 
Глобиш, Кшиштоф 
Ясная Поляна (Владимирская область) 
Западный мир 


# Network summary
- 